In [1]:
import os
import pickle

from collections import Counter

# import pandas as pd
from sklearn.metrics import classification_report

import numpy as np
import torch
import torch.nn as nn

import transformers
from transformers import Trainer
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers.data.data_collator import DataCollatorWithPadding

import datasets
from datasets import Dataset
from datasets import ClassLabel
from datasets import load_metric

## Global variables

In [2]:
# DATA_FOLDER = '/notebooks/Data/bert_sequence_classification'
DATA_FILE = 'emotion_analysis_comics/bert/datasets/comics_dataset.pt'
RESULTS_FOLDER = 'emotion_analysis_comics/bert/results'

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
device

device(type='cuda')

## Load data

In [5]:
dataset = torch.load(DATA_FILE)

/tmp/ipykernel_1529637/2065342137.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dataset = torch.load(DATA_FILE)


In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['file_name', 'page_nr', 'panel_nr', 'balloon_nr', 'utterance', 'raw_annotation', 'raw_emotion', 'raw_speaker_id', 'emotion', 'speaker_id', 'split', 'utterance_emotion', 'unique_emotion'],
        num_rows: 2804
    })
    test: Dataset({
        features: ['file_name', 'page_nr', 'panel_nr', 'balloon_nr', 'utterance', 'raw_annotation', 'raw_emotion', 'raw_speaker_id', 'emotion', 'speaker_id', 'split', 'utterance_emotion', 'unique_emotion'],
        num_rows: 1776
    })
    validation: Dataset({
        features: ['file_name', 'page_nr', 'panel_nr', 'balloon_nr', 'utterance', 'raw_annotation', 'raw_emotion', 'raw_speaker_id', 'emotion', 'speaker_id', 'split', 'utterance_emotion', 'unique_emotion'],
        num_rows: 702
    })
})

In [7]:
dataset['train']['utterance'][230]

'FOR AN EXPERT IN HEIST PLANNING, YOU SEEM TO HAVE A HARD TIME KEEPING YOUR PEOPLE IN CHECK.'

In [8]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

/Utilisateurs/umushtaq/.conda/envs/er_nb_env/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
label_names = list(set(dataset['train']['unique_emotion']))
label_nb = len(label_names)
labels = ClassLabel(num_classes=label_nb, names=label_names)

In [10]:
labels

ClassLabel(names=['surprise', 'joy', 'neutral', 'disgust', 'fear', 'anger', 'sadness'], id=None)

In [11]:
def tokenize(batch):
    tokens = tokenizer(batch['utterance'], truncation=True, padding=True, max_length=512)
    tokens['labels'] = labels.str2int(batch['unique_emotion'])
    return tokens

# this is just the text. if the results are nice, check transfer with text + topic 

In [12]:
dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/2804 [00:00<?, ? examples/s]

Map:   0%|          | 0/1776 [00:00<?, ? examples/s]

Map:   0%|          | 0/702 [00:00<?, ? examples/s]

In [13]:
dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['file_name', 'page_nr', 'panel_nr', 'balloon_nr', 'utterance', 'raw_annotation', 'raw_emotion', 'raw_speaker_id', 'emotion', 'speaker_id', 'split', 'utterance_emotion', 'unique_emotion', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2804
    })
    test: Dataset({
        features: ['file_name', 'page_nr', 'panel_nr', 'balloon_nr', 'utterance', 'raw_annotation', 'raw_emotion', 'raw_speaker_id', 'emotion', 'speaker_id', 'split', 'utterance_emotion', 'unique_emotion', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1776
    })
    validation: Dataset({
        features: ['file_name', 'page_nr', 'panel_nr', 'balloon_nr', 'utterance', 'raw_annotation', 'raw_emotion', 'raw_speaker_id', 'emotion', 'speaker_id', 'split', 'utterance_emotion', 'unique_emotion', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 702
    })
})

In [15]:
train_dataset = dataset['train'].shuffle(seed=42)
test_dataset = dataset['test'].shuffle(seed=42)
val_dataset = dataset['validation'].shuffle(seed=42)

In [16]:
dataset_d = {}
dataset_d['train'] = train_dataset
dataset_d['test'] = test_dataset
dataset_d['val'] = val_dataset

In [17]:
test_dataset

Dataset({
    features: ['file_name', 'page_nr', 'panel_nr', 'balloon_nr', 'utterance', 'raw_annotation', 'raw_emotion', 'raw_speaker_id', 'emotion', 'speaker_id', 'split', 'utterance_emotion', 'unique_emotion', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 1776
})

In [18]:
tokenizer.decode(dataset['train'][1945]['input_ids'])

'[CLS] oh! wait - wait wait! [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [19]:
# sanity check
set(dataset_d['train']['split'])

{'TRAIN'}

In [20]:
# sanity check
set(dataset_d['val']['split'])

{'TRAIN'}

In [21]:
# sanity check
set(dataset_d['test']['split'])

{'TEST'}

In [22]:
# global variables
NUM_LABELS = label_nb
BATCH_SIZE = 256
NB_EPOCHS = 100

In [23]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=NUM_LABELS)
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [24]:
model.device

device(type='cuda', index=0)

In [25]:
# https://huggingface.co/transformers/main_classes/trainer.html
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get('logits')
        loss_fct = nn.CrossEntropyLoss()#(weight=class_weights)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [26]:
metric = load_metric('f1', trust_remote_code=True)

def compute_metrics(eval_pred):
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    return metric.compute(predictions=predictions, references=labels, average='macro')

/tmp/ipykernel_1529637/2304026978.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('f1', trust_remote_code=True)


In [27]:
training_args = TrainingArguments(
    
    # output
    output_dir=RESULTS_FOLDER,          
    
    # params
    num_train_epochs=NB_EPOCHS,               # nb of epochs
    per_device_train_batch_size=BATCH_SIZE,   # batch size per device during training
    per_device_eval_batch_size=BATCH_SIZE,    # cf. paper Sun et al.
    learning_rate=1e-5,#2e-5,                 # cf. paper Sun et al.
#     warmup_steps=500,                         # number of warmup steps for learning rate scheduler
    warmup_ratio=0.1,                         # cf. paper Sun et al.
    weight_decay=0.01,                        # strength of weight decay
    
    # eval
    eval_strategy="steps",              # cf. paper Sun et al.
    eval_steps=20,                            # cf. paper Sun et al.
    
    # log
    logging_dir="emotion_analysis_comics/bert/logs",  
    logging_strategy='steps',
    logging_steps=20,
    
    # save
    save_strategy='steps',
    save_total_limit=1,
    # save_steps=20, # default 500
    load_best_model_at_end=True,              # cf. paper Sun et al.
    # metric_for_best_model='eval_loss' 
    metric_for_best_model='f1'
)

In [28]:
trainer = CustomTrainer( # Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

In [29]:
trainer.train()

/Utilisateurs/umushtaq/.conda/envs/er_nb_env/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,F1
20,1.917600,1.888249,0.045538
40,1.859100,1.803031,0.086193
60,1.742700,1.703001,0.159569
80,1.623700,1.643974,0.248808
100,1.507600,1.600837,0.283072
120,1.387000,1.564734,0.310935
140,1.268800,1.556541,0.325500
160,1.144500,1.561777,0.336556
180,1.020200,1.571944,0.345580
200,0.906800,1.601313,0.351120


/Utilisateurs/umushtaq/.conda/envs/er_nb_env/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/Utilisateurs/umushtaq/.conda/envs/er_nb_env/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/Utilisateurs/umushtaq/.conda/envs/er_nb_env/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/Utilisateurs/umushtaq/.conda/envs/er_nb_env/lib/python3.11/site-packages/to

TrainOutput(global_step=600, training_loss=0.70113037109375, metrics={'train_runtime': 272.6126, 'train_samples_per_second': 1028.566, 'train_steps_per_second': 2.201, 'total_flos': 1.282497856386e+16, 'train_loss': 0.70113037109375, 'epoch': 100.0})

In [30]:
# save best model
#trainer.save_model(os.path.join("/notebooks/cascade_bert/saved_models", 'best-model-with-real-prev-probs'))

In [31]:
#model_file = os.path.join("/notebooks/cascade_bert/saved_models", 'best-model-with-real-prev-probs')

#model = BertForSequenceClassification.from_pretrained(model_file, num_labels=NUM_LABELS)
#model.to(device)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [32]:
test_trainer = Trainer(model, data_collator=DataCollatorWithPadding(tokenizer))
test_raw_preds, test_labels, _ = test_trainer.predict(test_dataset)
test_preds = np.argmax(test_raw_preds, axis=1)

/Utilisateurs/umushtaq/.conda/envs/er_nb_env/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [33]:
len(test_preds)

1776

In [34]:
test_labels

array([0, 1, 0, ..., 1, 0, 5])

In [35]:
test_preds

array([4, 2, 4, ..., 6, 4, 5])

In [36]:
# labels=['fear', 'anger', 'disgust', 'joy', 'sadness', 'surprise', 'neutral']

In [37]:
target_name = labels.int2str([0,1,2,3,4,5,6])
print(classification_report(test_labels, test_preds, target_names=target_name, digits=3)) # type: ignore

              precision    recall  f1-score   support

    surprise      0.382     0.439     0.409       278
         joy      0.448     0.417     0.432       343
     neutral      0.216     0.085     0.122       129
     disgust      0.000     0.000     0.000        23
        fear      0.334     0.427     0.375       281
       anger      0.487     0.481     0.484       516
     sadness      0.251     0.267     0.259       206

    accuracy                          0.394      1776
   macro avg      0.303     0.302     0.297      1776
weighted avg      0.386     0.394     0.386      1776



/Utilisateurs/umushtaq/.conda/envs/er_nb_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Utilisateurs/umushtaq/.conda/envs/er_nb_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Utilisateurs/umushtaq/.conda/envs/er_nb_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,